In [1]:
import numpy as np 
import pandas as pd
from collections import Counter
from pandasql import sqldf
import re
import os 
os.environ['CUDA_VISIBLE_DEVICES']='1'
import joblib
import json 
import jieba 
import copy

from openai import AzureOpenAI
import matplotlib.pyplot as plt 
from sentence_transformers import SentenceTransformer
from FlagEmbedding import FlagReranker
from transformers import AutoTokenizer
from sklearn.metrics import average_precision_score

In [2]:
# pandasql查询函数需要的环境
pysqldf = lambda q: sqldf(q, globals())

In [3]:
# 原始数据处理
def format_model(x):
    model_list = x.split(',')
    model_list = [i.strip().lower() for i in model_list]
    new_list = [model_list[0]]
    i = 1
    while i < len(model_list):
        if (i != len(model_list) - 1) and (model_list[i-1] == model_list[i]):
            new_list.append(model_list[i]+model_list[i+1])
            if i < len(model_list) - 1:
                i += 2
            else:
                break
        elif (i != len(model_list) - 1) and (model_list[i-1] != model_list[i]):
            new_list.append(model_list[i])
            i += 1
        elif (model_list[i] == "上下水") or (model_list[i] == "air"):
            for j in range(len(new_list)):
                if model_list[i-1] == new_list[j]:
                    new_list.pop(j)
                    break
            new_list.append(model_list[i-1]+model_list[i])
            i += 1
        else:
            new_list.append(model_list[i])
            break
    return new_list

def format_all_models(x, dim_df):
    new_list = []
    for i in x:
        if i.find("全型号") >= 0:
            end_idx = i.find("全型号")
            name = i[:end_idx]
            new_list += [j for j in dim_df[dim_df['cat_name'] == name].model.tolist() if j not in x]
        else:
            new_list.append(i)
    return new_list

def format_series(x, dim_df):
    def contains_chinese(s):
        return re.search('[\u4e00-\u9fff]', s) is not None
    new_list = []
    for i in x:
        if i.find("系列") >= 0:
            end_idx = i.find("系列")
            name = i[:end_idx]
            new_list += [j for j in dim_df[(dim_df.model.str.find(name)>=0) & (
                dim_df.model.apply(lambda x: not contains_chinese(x)))].model.tolist() if j not in x]
            new_list += [i]
        else:
            new_list.append(i)
    return new_list

In [4]:
# 拼接openai embedding
def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [5]:
# 测试集处理及计算与正确qa的相似度
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = generate_embeddings(
        user_query,
    )
    df["similarities"] = df.ada_002.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )[["qa_id", "question", "answer", "similarities"]]
    return res.to_dict(orient='records')

def concat(x):
    return ",".join(x.astype(str).tolist())

def format_gt(x):
    if str(x) == "nan":
        return x
    else:
        return ",".join(x.split("\n"))

def count_gt(x):
    if str(x) == "nan":
        return 0
    else:
        return len(x.split(","))   

In [6]:
# 向量召回
def search_docs_bge(df, user_query, top_n=4, to_print=True, add_instruction=False, content_col="question"):
    df = df.copy()
    if add_instruction:
        instruction = "为这个句子生成表示以用于检索相关文章："
        user_query = instruction+user_query
    embedding = model.encode(user_query, normalize_embeddings=True).tolist()
    df = df.drop_duplicates([content_col])
    df["similarities"] = df.bge_large.apply(lambda x: cosine_similarity(x, embedding))
    output_columns = ["qa_id", "question", "answer", "similarities"]
    if "hit_reason" in df.columns:
        output_columns.append("hit_reason")
    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )[output_columns]
    return res.to_dict(orient='records')

def find_non_chinese_substrings(s):
    # 正则表达式解释：
    # [^\u4e00-\u9fff\W]+ 匹配非中文字符和非ASCII标点的连续字符
    # 但这样会排除空格，所以我们需要允许空格存在
    # 我们使用(?:[^\u4e00-\u9fff\W]| )+ 来实现这一点，(?:) 是非捕获组，用于匹配模式但不作为捕获结果返回
    # [^\u4e00-\u9fff\W] 匹配非中文且非标点的字符，| 表示或，空格 ' ' 被显式允许
    pattern = r'(?:[^\u4e00-\u9fff\W]| )+'
    
    # 使用findall方法查找所有匹配项
    matches = re.findall(pattern, s)
    
    # 过滤掉只包含空格的字符串
    matches = [match for match in matches if not match.isspace()]
    
    return matches

def clean_string(s):
    s = s.replace(" ", "").lower()
    return s

def find_model(x, all_model_list):
    x = x.replace("\n", "") 
    x = find_non_chinese_substrings(x)
    result = [clean_string(s) for s in x]
    return [model for model in all_model_list if model in result]

def find_cat(x, all_cat_list):
    return [name for name in all_cat_list if name in x]   

def filter_model(x, model_list):
    x = x.split(",")
    for model in model_list:
        if model in x:
            return True
    return False

def find_error_with_reason(a):
    # 第一次匹配“错误xxx”
    pattern1 = r"错误\s*\d+"
    matches1 = re.findall(pattern1, a)
    
    # 第二次匹配“错误原因xxx”
    pattern2 = r"错误原因\s*\d+"
    matches2 = re.findall(pattern2, a)

    # 合并两次匹配的结果
    matches = matches1 + matches2
    
    return [name.replace(" ", "").replace("原因", "") for name in matches]

def filter_reason(x, query_reason_list):
    reason_list = find_error_with_reason(x)
    for name in query_reason_list:
        if name in reason_list:
            return True 
    return False

def transform_model_name(x, all_model_list):
    x = x.replace("\n", "") 
    candidates = find_non_chinese_substrings(x)
    for name in candidates:
        cleaned_name = clean_string(name)
        for model in all_model_list:
            if cleaned_name == model:
                x = x.replace(name, model)
                break
    return x 

def remove_model_name(x, all_model_list):
    x = x.replace("\n", "") 
    candidates = find_non_chinese_substrings(x)
    for name in candidates:
        if clean_string(name) in all_model_list:
            x = x.replace(name, "")
    return x 

class BM25_Model(object):
    def __init__(self, documents_list, k1=2, k2=1, b=0.5):
        self.documents_list = documents_list
        self.documents_number = len(documents_list)
        self.avg_documents_len = sum([len(document) for document in documents_list]) / self.documents_number
        self.f = []
        self.idf = {}
        self.k1 = k1
        self.k2 = k2
        self.b = b
        self.init()

    def init(self):
        df = {}
        for document in self.documents_list:
            temp = {}
            for word in document:
                temp[word] = temp.get(word, 0) + 1
            self.f.append(temp)
            for key in temp.keys():
                df[key] = df.get(key, 0) + 1
        for key, value in df.items():
            self.idf[key] = np.log((self.documents_number - value + 0.5) / (value + 0.5))

    def get_score(self, index, query):
        score = 0.0
        document_len = len(self.f[index])
        qf = Counter(query)
        for q in query:
            if q not in self.f[index]:
                continue
            score += self.idf[q] * (self.f[index][q] * (self.k1 + 1) / (
                        self.f[index][q] + self.k1 * (1 - self.b + self.b * document_len / self.avg_documents_len))) * (
                                 qf[q] * (self.k2 + 1) / (qf[q] + self.k2))

        return score

    def get_documents_score(self, query, indices):
        score_list = []
        for i in indices:
            score_list.append(self.get_score(i, query))
        return score_list


class WordCut:
    def __init__(self, all_model_list=None):
        with open('/data/dataset/kefu/hit_stopwords.txt', encoding='utf-8') as f: # 可根据需要打开停用词库，然后加上不想显示的词语
            con = f.readlines()
            stop_words = set()
            for i in con:
                i = i.replace("\n", "")   # 去掉读取每一行数据的\n
                stop_words.add(i)
        self.stop_words = stop_words
        self.all_model_list = all_model_list
        
    def cut(self, mytext):
        # jieba.load_userdict('自定义词典.txt')  # 这里你可以添加jieba库识别不了的网络新词，避免将一些新词拆开
        # jieba.initialize()  # 初始化jieba
        # 文本预处理 ：去除一些无用的字符只提取出中文出来
        # new_data = re.findall('[\u4e00-\u9fa5]+', mytext, re.S)
        # new_data = " ".join(new_data)
        # 匹配中英文标点符号，以及全角和半角符号
        pattern = r'[\u3000-\u303f\uff01-\uff0f\uff1a-\uff20\uff3b-\uff40\uff5b-\uff65\u2018\u2019\u201c\u201d\u2026\u00a0\u2022\u2013\u2014\u2010\u2027\uFE10-\uFE1F\u3001-\u301E]|[\.,!¡?¿\-—_(){}[\]\'\";:/]'
        # 使用 re.sub 替换掉符合模式的字符为空字符
        new_data = re.sub(pattern, '', mytext)
        new_data = transform_model_name(new_data, self.all_model_list)
        # 文本分词
        seg_list_exact = jieba.lcut(new_data)
        result_list = []
        # 去除停用词并且去除单字
        for word in seg_list_exact:
            if word not in self.stop_words and len(word) > 1:
                result_list.append(word) 
        return result_list

def search_docs_bm25(df, indices, user_query, top_n=4):
    # document_list = [wc.cut(doc) for doc in df.question]
    # bm25_model = BM25_Model(document_list)
    embedding = wc.cut(user_query)
    df["similarities"] = bm25_model.get_documents_score(embedding, indices)
    output_columns = ["qa_id", "question", "answer", "similarities"]
    if "hit_reason" in df.columns:
        output_columns.append("hit_reason")
    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )[output_columns]
    return res.to_dict(orient='records')

In [7]:
# 综合分析
def ranking_metric(x):
    if (x.find("error")>=0) and (x.find("model")>=0):
        return 1 
    elif (x.find("error")>=0) and (x.find("cat")>=0):
        return 2 
    elif (x.find("error")>=0):
        return 3 
    elif (x.find("model")>=0):
        return 4
    elif (x.find("cat")>=0):
        return 5
    else:
        return 6

def mine_hard_negative(x, similarities, reason, result, positive):
    positives = x[positive].split(",")
    df = pd.DataFrame(x[[similarities, reason, result]].to_dict())
    df = df[~df[result].isin(positives)]
    df["ranking"] = df[reason].apply(lambda x: ranking_metric(x))
    df = df.sort_values(["ranking", similarities], ascending=[True, False])
    df = df.drop_duplicates(result)
    df = df.iloc[:10]
    return pd.Series({col+"_hard": df[col].values.tolist() for col in [similarities, reason, result]})

def format_result(x, similarities, reason, result, contents):
    assert len(x[contents]) == len(x[result])
    num_result = len(x[result])
    new_set = dict()
    for j in range(num_result):
        content = x[contents][j]
        result_name = x[result][j]
        sim = x[similarities][j]
        reason_code = x[reason][j]
        if content in new_set:
            if (ranking_metric(reason_code) <= ranking_metric(new_set[content]["reason"])
               ) & (sim > new_set[content]["similarities"]):
                new_set.update({content: {"similarities": sim, "reason": reason_code, "result": result_name}})
        else:
            new_set.update({content: {"similarities": sim, "reason": reason_code, "result": result_name}})
    df = pd.DataFrame(new_set).T.reset_index().rename(columns={"index": "content"})
    df["ranking"] = df["reason"].apply(lambda x: ranking_metric(x))
    df = df.sort_values(["ranking", "similarities"], ascending=[True, False])
    return df[['result', 'reason', 'similarities', 'content']].to_dict(orient='records')

def merge_recall(x, recall_list, weights):
    pool = {}
    for i in range(len(weights)):
        weight = weights[i] 
        recall_name = recall_list[i]
        num_results = len(x[recall_name])
        for j in range(num_results):
            result_item = x[recall_name][j]
            result = result_item["result"]
            if result in pool:
                if ranking_metric(result_item['reason']) < ranking_metric(pool[result]['reason']):
                    reason = result_item['reason']
                    pool[result]['reason'] = reason
                pool[result]['similarities'] += weight * result_item['similarities'] / sum(weights)
                pool[result]['full_reason'] = pool[result]['full_reason']+","+result_item['reason']+"_"+recall_name
            else:
                pool[result]= {
                    "reason": result_item['reason'],
                    "similarities": weight * result_item['similarities'] / sum(weights),
                    "full_reason": result_item['reason']+"_"+recall_name
                              }
    df = pd.DataFrame(pool).T.reset_index().rename(columns={"index": "result"})
    df["ranking"] = df["reason"].apply(lambda x: ranking_metric(x))
    df = df.sort_values(["ranking", "similarities"], ascending=[True, False])
    return df[['result', 'reason', 'full_reason', 'similarities']].to_dict(orient='records')

def mine_hard_negative2(x, recall, top_n, positive, output_cols):
    df = pd.DataFrame(x[recall])
    positives = x[positive].split(",")
    df = df[~df["result"].isin(positives)]
    df["ranking"] = df["reason"].apply(lambda x: ranking_metric(x))
    df = df.sort_values(["ranking", "similarities"], ascending=[True, False])
    df = df.iloc[:top_n]
    return df[output_cols].to_dict(orient='records')  

def split_recall(x, output_cols, new_cols):
    df = pd.DataFrame(x)
    return pd.Series({new_col: df[col].values.tolist() 
                      for col, new_col in zip(output_cols, new_cols)})

def find_score_limit(x):
    min_all = float("inf")
    max_all = float("-inf")
    for i in range(len(x)):
        min_i = min(x[i])
        max_i = max(x[i])
        min_all = min(min_all, min_i)
        max_all = max(max_all, max_i)
    return min_all, max_all

def convert_limit(x, min_all, max_all):
    return [(i-min_all)/(max_all-min_all) for i in x]

def convert_df_to_jsonl(df, filename, query="question_cleaned", pos_col="question_positive", neg_col="question_bge_hard"):
    with open(filename, 'w') as file:
        for _, row in df.iterrows():
            # Constructing the dictionary for each row
            data = {
                "query": row[query],
                "pos": row[pos_col],
                "neg": row[neg_col]
            }
            # Writing the JSON string followed by a newline character to make it JSONL
            file.write(json.dumps(data) + '\n')

In [8]:
# 排序
def mrr_at_k_score(is_relevant, pred_ranking, k):
    """
    Computes MRR@k score

    Args:
        is_relevant (`List[bool]` of length `num_pos+num_neg`): True if the document is relevant
        pred_ranking (`List[int]` of length `num_pos+num_neg`): Indices of the documents sorted in decreasing order
            of the similarity score

    Returns:
        mrr_score (`float`): MRR@k score
    """
    mrr_score = 0
    for rank, index in enumerate(pred_ranking[:k]):
        if is_relevant[index]:
            mrr_score = 1 / (rank + 1)
            break

    return mrr_score

def recall_at_k_score(is_relevant, pred_ranking, k):
    """
    Computes MRR@k score

    Args:
        is_relevant (`List[bool]` of length `num_pos+num_neg`): True if the document is relevant
        pred_ranking (`List[int]` of length `num_pos+num_neg`): Indices of the documents sorted in decreasing order
            of the similarity score

    Returns:
        mrr_score (`float`): MRR@k score
    """
    recall_score = 0
    for index in pred_ranking[:k]:
        if is_relevant[index]:
            recall_score = 1
            break

    return recall_score

def ap_score(is_relevant, pred_scores):
    """
    Computes AP score

    Args:
        is_relevant (`List[bool]` of length `num_pos+num_neg`): True if the document is relevant
        pred_scores (`List[float]` of length `num_pos+num_neg`): Predicted similarity scores

    Returns:
        ap_score (`float`): AP score
    """
    # preds = np.array(is_relevant)[pred_scores_argsort]
    # precision_at_k = np.mean(preds[:k])
    # ap = np.mean([np.mean(preds[: k + 1]) for k in range(len(preds)) if preds[k]])
    ap = average_precision_score(is_relevant, pred_scores)
    return ap

def compute_recall_score(df, model, query, recall):
    pairs = []
    for i in range(df.shape[0]):
        sample = df.iloc[i]
        for p in sample[recall]:
            pairs.append([sample[query], p])
    all_scores = model.compute_score(pairs)
    result = []
    start_inx = 0
    for i in range(df.shape[0]):
        sample = df.iloc[i]
        pred_scores = all_scores[start_inx:start_inx + len(sample[recall])]
        result.append(pred_scores)
        start_inx += len(sample[recall])
    return result

def compute_metrics_batched_from_crossencoder(df, score, relevant, 
                                              mrr_at_k=10, recall_at_list=[1,2], metrics=["map", "mrr", "recall"]):
    all_mrr_scores = []
    all_ap_scores = []
    all_recall_scores = [[] for _ in range(len(recall_at_list))]

    for i in range(df.shape[0]):
        sample = df.iloc[i]
        is_relevant = sample[relevant]
        pred_scores = np.array(sample[score])

        pred_scores_argsort = np.argsort(-pred_scores)  # Sort in decreasing order
        if "mrr" in metrics:
            mrr = mrr_at_k_score(is_relevant, pred_scores_argsort, mrr_at_k)
            all_mrr_scores.append(mrr)
        if "map" in metrics:
            ap = ap_score(is_relevant, pred_scores)
            all_ap_scores.append(ap)
        if "recall" in metrics:
            for recall_index, recall_at in enumerate(recall_at_list):
                recall_score = recall_at_k_score(is_relevant, pred_scores_argsort, recall_at)
                all_recall_scores[recall_index].append(recall_score)

    result = {}
    if "map" in metrics:
        mean_ap = np.mean(all_ap_scores)
        result["map"] = mean_ap
    if "mrr" in metrics:
        mean_mrr = np.mean(all_mrr_scores)
        result[f"mrr@{mrr_at_k}"] = mean_mrr
    if "recall" in metrics:
        for recall_index, recall_at in enumerate(recall_at_list):
            result[f"recall@{recall_at}"] = np.mean(all_recall_scores[recall_index])
    return result

def find_T_loc(x, relevant, score):
    is_relevant = x[relevant]
    pred_scores = np.array(x[score])
    pred_scores_argsort = np.argsort(-pred_scores)
    for rank, index in enumerate(pred_scores_argsort):
        if is_relevant[index]:
            return rank
    return np.nan

def get_reranking(x, relevant, score, recall):
    is_relevant = x[relevant]
    pred_scores = np.array(x[score])
    pred_scores_argsort = np.argsort(-pred_scores)
    recall_list = copy.deepcopy(x[recall])
    reranking = []
    for i in pred_scores_argsort:
        temp = recall_list[i]
        temp.update({"ranking_score": pred_scores[i]})
        reranking.append(temp)
    return reranking

# 向量召回

In [9]:
dim_df = pd.read_csv("/data/dataset/kefu/dim_df20240315.csv")
all_model_list = dim_df.model.tolist()
all_cat_list = dim_df.cat_name.unique().tolist()

In [10]:
oot = pd.read_csv("/data/dataset/kefu/oot20240315.csv")

In [11]:
test = pd.read_csv("/data/dataset/kefu/test20240315.csv")
# df1 = pd.read_csv("/data/dataset/kefu/database_before_online_with_emb.csv")
# df2 = pd.read_csv("/data/dataset/kefu/database_with_emb20240315.csv")

In [12]:
df1 = pd.read_csv("/data/dataset/kefu/database_before_online.csv")
df2 = pd.read_csv("/data/dataset/kefu/database20240506.csv")

In [13]:
df1_vedio = df1[df1.answer.apply(lambda x: True if x.lower().strip()[:10].find("http")>=0 else False)]
df1_fig = df1[df1.question.apply(lambda x: True if x.lower().strip().find("开箱图")>=0 else False)]
df2_vedio = df2[df2.answer.apply(lambda x: True if x.lower().strip()[:10].find("http")>=0 else False)]
df2_fig = df2[df2.question.apply(lambda x: True if x.lower().strip().find("开箱图")>=0 else False)]

In [14]:
df1 = df1[df1.answer.apply(lambda x: True if x.lower().strip()[:10].find("http")<0 else False)]
df1 = df1[df1.question.apply(lambda x: True if x.lower().strip().find("开箱图")<0 else False)]

In [15]:
df2 = df2[df2.answer.apply(lambda x: True if x.lower().strip()[:10].find("http")<0 else False)]
df2 = df2[df2.question.apply(lambda x: True if x.lower().strip().find("开箱图")<0 else False)]

In [16]:
df2 = df2.drop("error_list", axis=1)

In [17]:
df1.shape

(1448, 12)

In [18]:
df2.shape

(2883, 12)

In [19]:
df = pd.concat([df1, df2], axis=0)

In [20]:
df = df.sort_values(["question", "model", "update_time"],
                       ).drop_duplicates(subset=["question", "model"], keep='last')

In [21]:
df.shape

(3276, 12)

In [22]:
model = SentenceTransformer('/data/dataset/huggingface/hub/bge-large-zh-v1.5')
# model = SentenceTransformer('/workspace/data/private/zhuxiaohai/models/bge_finetune_emb')
# q_embeddings = model.encode(df1.question.tolist(), normalize_embeddings=True, batch_size=32)
# df1['bge_large'] = q_embeddings.tolist()
# q_embeddings = model.encode(df2.question.tolist(), normalize_embeddings=True, batch_size=32)
# df2['bge_large'] = q_embeddings.tolist()

In [23]:
# 标签+向量3

In [24]:
q_embeddings = model.encode(df.answer.tolist(), normalize_embeddings=True, batch_size=32)
df['bge_large'] = q_embeddings.tolist()

In [24]:
test = df.copy()
df1 = df.copy()
test["gt_qa_id"] = test['qa_id']

In [191]:
def sub_worker(result, score, reason, content, top_n, content_col):
    if (filter_mask & (reason_indicator.str.find("errorcode")>=0)).sum() > 0:
        aug_mask = filter_mask & (reason_indicator.str.find("errorcode")>=0)
        filtered_df = df1[aug_mask].copy()
        filtered_df["hit_reason"] = reason_indicator[aug_mask].copy()
        res = search_docs_bge(filtered_df, question, top_n=top_n, add_instruction=True, content_col=content_col)
        result += [j["qa_id"] for j in res]
        score += [round(j["similarities"], 2) for j in res]
        reason += [j["hit_reason"] for j in res]
        content += [j[content_col] for j in res]
        
        if len(set(content)) < top_n:
            aug_mask = filter_mask & (~(reason_indicator.str.find("errorcode")>=0))
            filtered_df = df1[aug_mask].copy()
            filtered_df["hit_reason"] = reason_indicator[aug_mask].copy()
            res = search_docs_bge(filtered_df, question, top_n=top_n, add_instruction=True, content_col=content_col)
            result += [j["qa_id"] for j in res]
            score += [round(j["similarities"], 2) for j in res]
            reason += [j["hit_reason"] for j in res]
            content += [j[content_col] for j in res]
    else:
        aug_mask = (reason_indicator.str.find("errorcode")>=0)
        if aug_mask.sum() > 0:
            filtered_df = df1[aug_mask].copy()
            filtered_df["hit_reason"] = reason_indicator[aug_mask].copy()
            res = search_docs_bge(filtered_df, question, top_n=top_n, add_instruction=True, content_col=content_col)
            result += [j["qa_id"] for j in res]
            score += [round(j["similarities"], 2) for j in res]
            reason += [j["hit_reason"] for j in res]
            content += [j[content_col] for j in res]

        if len(set(content)) < top_n:
            aug_mask = filter_mask
            filtered_df = df1[aug_mask].copy()
            filtered_df["hit_reason"] = reason_indicator[aug_mask].copy()
            res = search_docs_bge(filtered_df, question, top_n=top_n, add_instruction=True, content_col=content_col)
            result += [j["qa_id"] for j in res]
            score += [round(j["similarities"], 2) for j in res]
            reason += [j["hit_reason"] for j in res]
            content += [j[content_col] for j in res]

    if len(set(content)) < top_n:
        aug_mask = (~filter_mask) & (reason_indicator.str.find("cat")>=0)
        if aug_mask.sum() > 0:
            filtered_df = df1[aug_mask].copy()
            filtered_df["hit_reason"] = reason_indicator[aug_mask].copy()
            res = search_docs_bge(filtered_df, question, top_n=top_n, add_instruction=True, content_col=content_col)
            result += [j["qa_id"] for j in res]
            score += [round(j["similarities"], 2) for j in res]
            reason += [j["hit_reason"] for j in res]
            content += [j[content_col] for j in res]
    return result, score, reason, content
    
label = []
result_list = []
top_n = 20
for i in range(test.shape[0]):
    if i % 10 ==0:
        print(i)
    gt = test['gt_qa_id'].iloc[i].split(",")
    question = test['question'].iloc[i]
    model_list = find_model(question, all_model_list)
    cat_list = find_cat(question, all_cat_list)   
    cat_list += [cat for cat in dim_df.loc[dim_df.model.isin(model_list), 'cat_name'].tolist() if cat not in cat_list]
    reason_list = find_error_with_reason(question)
    model_mask = (df1.model_list.apply(lambda x: filter_model(x, model_list)))
    cat_mask = (df1.cat_name.apply(lambda x: filter_model(x, cat_list)))
    reason_mask = (df1.question.apply(lambda x: filter_reason(x, reason_list)))
    reason_indicator = pd.Series(["none"]*df1.shape[0], index=df1.index)
    reason_indicator[model_mask] = reason_indicator[model_mask].apply(lambda x: x + "|model" if x != "none" else "model")
    reason_indicator[cat_mask] = reason_indicator[cat_mask].apply(lambda x: x + "|cat" if x != "none" else "cat")
    reason_indicator[reason_mask] = reason_indicator[reason_mask].apply(lambda x: x + "|errorcode" if x != "none" else "errorcode")
    result = []
    score = []
    reason = []
    content = []
    content_col = "answer"
    # question = remove_model_name(question, all_model_list)
    filter_mask = (reason_indicator.str.find("model")>=0)
    if filter_mask.sum() > 0:
        result, score, reason, content = sub_worker(result, score, reason, content, top_n, "answer")
    else:
        filter_mask = (reason_indicator.str.find("cat")>=0)   
        if filter_mask.sum() > 0:
            result, score, reason, content = sub_worker(result, score, reason, content, top_n, "answer")
    if len(result) == 0:
        filter_mask = (reason_indicator.str.find("errorcode")>=0)
        if filter_mask.sum() > 0:
            aug_mask = filter_mask
            filtered_df = df1[aug_mask].copy()
            filtered_df["hit_reason"] = reason_indicator[aug_mask].copy()
            res = search_docs_bge(filtered_df, question, top_n=top_n, add_instruction=True, content_col=content_col)
            result += [j["qa_id"] for j in res]
            score += [round(j["similarities"], 2) for j in res]
            reason += [j["hit_reason"] for j in res]  
            content += [j[content_col] for j in res]
            if len(set(content)) < top_n:
                aug_mask = (~filter_mask)
                filtered_df = df1[aug_mask].copy()
                filtered_df["hit_reason"] = reason_indicator[aug_mask].copy()
                res = search_docs_bge(filtered_df, question, top_n=top_n, add_instruction=True, content_col=content_col)
                result += [j["qa_id"] for j in res]
                score += [round(j["similarities"], 2) for j in res]
                reason += [j["hit_reason"] for j in res]  
                content += [j[content_col] for j in res]
        else:
            filtered_df = df1.copy()
            filtered_df["hit_reason"] = reason_indicator.copy()
            res = search_docs_bge(filtered_df, question, top_n=top_n, add_instruction=True, content_col=content_col)
            result += [j["qa_id"] for j in res]
            score += [round(j["similarities"], 2) for j in res]
            reason += [j["hit_reason"] for j in res]
            content += [j[content_col] for j in res]
    
    found = False
    for j in result:
        if j in gt:
            found = True
            break 
    if found:
        label.append(1)
    else:
        label.append(0)
    result_list.append({"qa_id": test['qa_id'].iloc[i], 
                        "result": result, 
                        "similarities": score, 
                        "hit_reason": reason, 
                        "content": content,
                        "label": int(found)})

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [26]:
# joblib.dump(result_list, "/data/dataset/kefu/query_answer_sim.json")

In [25]:
result_list = joblib.load("/data/dataset/kefu/query_answer_sim.json")

In [26]:
result_df = pd.DataFrame(result_list)
test_result = test.merge(result_df, how='left', left_on='qa_id', right_on='qa_id')
test_result["sim_max"] = test_result["similarities"].apply(lambda x: max(x))
test_result["sim_min"] = test_result["similarities"].apply(lambda x: min(x))
test_result["result_num"] = test_result.result.apply(lambda x: len(x))

In [27]:
# test_result = test_result.sort_values(["question", "model", "update_time"],
#                        ).drop_duplicates(subset=["question", "model"], keep='last')

In [28]:
test_result.label.mean()

0.6242368742368742

# 综合分析

In [29]:
final_result = test_result.rename(columns={col: col+"_bge" for col in [
                                               "result",
                                               "similarities",
                                               "hit_reason",
                                               "content",
                                               "label",
                                               "sim_max",
                                               "sim_min",
                                               "result_num"]})

In [30]:
# final_result = pd.merge(left=test_result, right=test_result2[["qa_id",
#                                                "result",
#                                                "similarities",
#                                                "hit_reason",
#                                                "label",
#                                                "sim_max",
#                                                "sim_min",
#                                                "result_num"]], 
#                  left_on="qa_id", right_on="qa_id", how="left",
#                  suffixes=["_bge", "_bm25"])

In [31]:
# final_result['label_all'] = final_result[["label_bge", "label_bm25"]].apply(lambda x: max(x["label_bge"], x["label_bm25"]), axis=1)

In [32]:
# min_all, max_all = find_score_limit(final_result["similarities_bm25"].tolist())
# final_result["similarities_rescaled_bm25"] = final_result["similarities_bm25"].apply(
#     lambda x: convert_limit(x, min_all, max_all))

In [33]:
# cols = ["similarities_bge", "hit_reason_bge", "result_bge"]
# final_result["recall_bge"] = final_result[cols].apply(lambda x: format_result(x, *cols), axis=1) 
# cols = ["similarities_rescaled_bm25", "hit_reason_bm25", "result_bm25"]
# final_result["recall_bm25"] = final_result[cols].apply(lambda x: format_result(x, *cols), axis=1) 
# final_result["recall_all"] = final_result[["recall_bge", "recall_bm25"]].apply(
#     lambda x: merge_recall(x, 
#                            recall_list=["recall_bge", "recall_bm25"], 
#                            weights=[0.9, 0.84]
#                           ), 
#     axis=1) 

In [34]:
cols = ["similarities_bge", "hit_reason_bge", "result_bge", "content_bge"]
final_result["recall_bge"] = final_result[cols].apply(lambda x: format_result(x, *cols), axis=1) 

In [35]:
(final_result["recall_bge"].apply(
    lambda x: len([i["content"] for i in x])) != final_result["recall_bge"].apply(
    lambda x: len(set([i["content"] for i in x])))).sum()

0

In [36]:
def count_model_reason(x):
    count = 0
    for item in x:
        i = item["reason"]
        if (i.find("model") >= 0) & (i.find("error") < 0):
            count += 1
    return count 

def count_error_reason(x):
    count = 0
    for item in x:
        i = item["reason"]
        if (i.find("model") < 0) & (i.find("cat") < 0) & (i.find("error") >= 0):
            count += 1
    return count 

def count_cat_reason(x):
    count = 0
    for item in x:
        i = item["reason"]
        if (i.find("cat") >= 0) & (i.find("model") < 0) & (i.find("error") < 0):
            count += 1
    return count 

def count_none_reason(x):
    count = 0
    for item in x:
        i = item["reason"]
        if (i.find("cat") < 0) & (i.find("model") < 0) & (i.find("error") < 0):
            count += 1
    return count 

def get_gt_pos(x, gt_col, recall_col):
    positives = x[gt_col].split(",")
    for index, item in enumerate(x[recall_col]):
        i = item["result"]
        if i in positives:
            return index
    return np.nan

In [37]:
final_result["reason_model_num"] = final_result["recall_bge"].apply(lambda x: count_model_reason(x))
final_result["reason_error_num"] = final_result["recall_bge"].apply(lambda x: count_error_reason(x))
final_result["reason_cat_num"] = final_result["recall_bge"].apply(lambda x: count_cat_reason(x))
final_result["reason_none_num"] = final_result["recall_bge"].apply(lambda x: count_none_reason(x))
final_result["result_unique_num"] = final_result["recall_bge"].apply(lambda x: len(x))
final_result["gt_pos"] = final_result[["recall_bge", "gt_qa_id"]].apply(lambda x: get_gt_pos(x, "gt_qa_id", "recall_bge"), axis=1)

In [221]:
final_result.agg({"reason_model_num": [len, "min", "max", "mean", "median"],
                  "reason_error_num": [len, "min", "max", "mean", "median"],
                  "reason_cat_num": [len, "min", "max", "mean", "median"],
                  "reason_none_num": [len, "min", "max", "mean", "median"],
                  "result_unique_num": [len, "min", "max", "mean", "median"],
                  "gt_pos": [len, "min", "max", "mean", "median"]})

,reason_model_num,reason_error_num,reason_cat_num,reason_none_num,result_unique_num,gt_pos
len,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000
min,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000
max,20.000000,7.000000,20.000000,20.000000,27.000000,19.000000
mean,1.593407,0.058303,2.466422,15.939866,20.065934,2.935452
median,0.000000,0.000000,0.000000,20.000000,20.000000,1.000000


In [222]:
final_result[final_result["reason_none_num"]>0].agg({"reason_none_num": [len, "min", "max", "mean", "median"],
                                                      "result_unique_num": [len, "min", "max", "mean", "median"],
                                                      "gt_pos": [len, "min", "max", "mean", "median"]})

,reason_none_num,result_unique_num,gt_pos
len,2611.000000,2611.000000,2611.000000
min,19.000000,20.000000,0.000000
max,20.000000,27.000000,19.000000
mean,19.999617,20.072769,3.073983
median,20.000000,20.000000,1.000000


In [223]:
final_result[final_result["reason_model_num"]>0].agg({"reason_model_num": [len, "min", "max", "mean", "median"],
                                                      "result_unique_num": [len, "min", "max", "mean", "median"],
                                                      "gt_pos": [len, "min", "max", "mean", "median"]})

,reason_model_num,result_unique_num,gt_pos
len,261.0,261.0,261.000000
min,20.0,20.0,0.000000
max,20.0,20.0,19.000000
mean,20.0,20.0,2.009756
median,20.0,20.0,0.000000


In [224]:
final_result[final_result["reason_error_num"]>0].agg({"reason_error_num": [len, "min", "max", "mean", "median"],
                                                      "result_unique_num": [len, "min", "max", "mean", "median"],
                                                      "gt_pos": [len, "min", "max", "mean", "median"]})

,reason_error_num,result_unique_num,gt_pos
len,79.000000,79.000000,79.000000
min,1.000000,21.000000,0.000000
max,7.000000,27.000000,4.000000
mean,2.417722,22.405063,0.655172
median,2.000000,22.000000,0.000000


In [225]:
final_result[final_result["reason_cat_num"]>0].agg({"reason_cat_num": [len, "min", "max", "mean", "median"],
                                                      "result_unique_num": [len, "min", "max", "mean", "median"],
                                                      "gt_pos": [len, "min", "max", "mean", "median"]})

,reason_cat_num,result_unique_num,gt_pos
len,404.0,404.000000,404.000000
min,20.0,20.000000,0.000000
max,20.0,23.000000,19.000000
mean,20.0,20.064356,2.775229
median,20.0,20.000000,1.000000


In [110]:
# merge_target = "recall_bge"
# output_cols = ["result", "reason", "similarities"]
# final_result["recall_hard"] = final_result[[merge_target, "gt_qa_id"]].apply(
#     lambda x: mine_hard_negative2(x, 
#                                   merge_target, 
#                                   positive='gt_qa_id', 
#                                   top_n=20,
#                                   output_cols=output_cols
#                                  ), 
#     axis=1) 

In [39]:
cols = ["similarities", "reason", "result"]
new_cols = ["similarities_merge_hard", "reason_merge_hard", "result_merge_hard"]
# cols = ["similarities", "reason", "result"]
# new_cols = ["similarities_merge_hard", "reason_merge_hard", "result_merge_hard"]
final_result[new_cols] = final_result["recall_bge"].apply(lambda x: split_recall(x, cols, new_cols))

In [40]:
# final_result['result_num_bge_set'] = final_result.recall_bge.apply(lambda x: len(x))
# final_result['result_num_bm25_set'] = final_result.recall_bm25.apply(lambda x: len(x))
# final_result['result_num_all_set'] = final_result.recall_all.apply(lambda x: len(x))

In [41]:
# 制作数据集

In [42]:
temp = final_result.copy()
temp_exploded = temp.explode("result_merge_hard")[['qa_id', 'result_merge_hard']]
temp_right = df1[['qa_id', 
               'question', 
               'answer']].copy()
temp_exploded = pd.merge(left=temp_exploded, right=temp_right, 
                         left_on='result_merge_hard', right_on='qa_id', 
                         how='left', suffixes=["", "_merge_hard"])[["qa_id", "result_merge_hard", "question", "answer"]]
temp_exploded = temp_exploded.rename(columns={"question": "question_merge_hard", "answer": "answer_merge_hard"})
temp_exploded = temp_exploded.groupby("qa_id")[["question_merge_hard", "answer_merge_hard"]].apply(
    lambda x: pd.Series({col: x[col].tolist() for col in x.columns}))


In [43]:
final_result = pd.merge(left=final_result, right=temp_exploded,
                        left_on='qa_id', right_on='qa_id', how='left')

In [44]:
# temp = final_result.copy()
# temp_exploded = temp.explode("result_bge_hard")[['qa_id', 'result_bge_hard']]
# temp_right = df1[['qa_id', 
#                'question', 
#                'answer']].copy()
# temp_exploded = pd.merge(left=temp_exploded, right=temp_right, 
#                          left_on='result_bge_hard', right_on='qa_id', 
#                          how='left', suffixes=["", "_bge_hard"])[["qa_id", "result_bge_hard", "question", "answer"]]
# temp_exploded = temp_exploded.rename(columns={"question": "question_bge_hard", "answer": "answer_bge_hard"})
# temp_exploded = temp_exploded.groupby("qa_id")[["question_bge_hard", "answer_bge_hard"]].apply(
#     lambda x: pd.Series({col: x[col].tolist() for col in x.columns}))

In [45]:
# final_result = pd.merge(left=final_result, right=temp_exploded,
#                         left_on='qa_id', right_on='qa_id', how='left')

In [46]:
temp = final_result.copy()
temp['gt_qa_id'] = temp['gt_qa_id'].apply(lambda x: x.split(","))
temp_exploded = temp.explode("gt_qa_id")[['qa_id', 'gt_qa_id']]
temp_right = df1[['qa_id', 
               'question', 
               'answer']].copy()
temp_exploded = pd.merge(left=temp_exploded, right=temp_right, 
                         left_on='gt_qa_id', right_on='qa_id', 
                         how='left', suffixes=["", "_kg"])[["qa_id", "gt_qa_id", "question", "answer"]]
temp_exploded = temp_exploded.rename(columns={"question": "question_positive", "answer": "answer_positive"})
temp_exploded = temp_exploded.groupby("qa_id")[["question_positive", "answer_positive"]].apply(
    lambda x: pd.Series({col: x[col].tolist() for col in x.columns}))
final_result = pd.merge(left=final_result, right=temp_exploded,
                        left_on='qa_id', right_on='qa_id', how='left')

In [47]:
import random
import faiss
import tqdm
def create_index(embeddings, use_gpu):
    index = faiss.IndexFlatIP(len(embeddings[0]))
    embeddings = np.asarray(embeddings, dtype=np.float32)
    if use_gpu:
        co = faiss.GpuMultipleClonerOptions()
        co.shard = True
        co.useFloat16 = True
        index = faiss.index_cpu_to_all_gpus(index, co=co)
    index.add(embeddings)
    return index

def batch_search(index,
                 query,
                 topk: int = 200,
                 batch_size: int = 64):
    all_scores, all_inxs = [], []
    for start_index in range(0, len(query), batch_size):
        batch_query = query[start_index:start_index + batch_size]
        batch_scores, batch_inxs = index.search(np.asarray(batch_query, dtype=np.float32), k=topk)
        all_scores.extend(batch_scores.tolist())
        all_inxs.extend(batch_inxs.tolist())
    return all_scores, all_inxs

In [48]:
corpus = list(set(df.answer.tolist()))
p_vecs = model.encode(corpus, normalize_embeddings=True, batch_size=32)

In [49]:
print('create index and search------------------')
index = create_index(p_vecs, use_gpu=True)

create index and search------------------


In [50]:
instruction = "为这个句子生成表示以用于检索相关文章："

In [51]:
q_vecs =  model.encode([instruction+i for i in final_result["question"].tolist()], normalize_embeddings=True, batch_size=32)

In [52]:
final_result["query_vec"] = q_vecs.tolist()

In [53]:
def mine_hard_negative3(x, query, recall, positive, embedding, top_range, negative_number):
    question = x[query]
    positive_list = x[positive]
    start, end = top_range
    negative_list = []
    recall_list = []
    for item in x[recall]:
        if item not in recall_list:
            recall_list.append(item)
    for item in recall_list[start:end]:
        if (item not in positive_list) and (item != question):
            negative_list.append(item)
    recall_list = [item for item in recall_list if (item not in negative_list) & (item != question)]
    if len(negative_list) > negative_number:
        negative_list = random.sample(negative_list, negative_number)
    elif (len(negative_list) < negative_number) & (len(recall_list) >= (negative_number - len(negative_list) + len(positive_list))):
        samples = random.sample(recall_list, negative_number - len(negative_list) + len(positive_list))
        samples = [sent for sent in samples if sent not in positive_list]
        negative_list.extend(samples[:negative_number - len(negative_list)])
    else:
        _, a = batch_search(index, [x[embedding]], topk=negative_number + len(positive_list)+1)
        samples = [corpus[i] for i in a[0]]
        samples = [sent for sent in samples if (sent not in positive_list) & (sent not in negative_list) & (sent != query)]
        negative_list.extend(samples[:negative_number - len(negative_list)])
    return negative_list

In [54]:
random.seed(1024)
final_result["answer_negative"] = final_result.apply(lambda x: mine_hard_negative3(
    x, "question", "answer_merge_hard", "answer_positive", "query_vec", [10,20], 5), axis=1)

In [55]:
(final_result["answer_negative"].apply(
    lambda x: len(x)) != final_result["answer_negative"].apply(lambda x: len(set(x)))).sum()

0

In [56]:
# 用来训练bge ranker模型的数据集（采用finetuned embedding模型召回后，每路召回取前10个，融合取前20个）
# Replace 'your_dataframe.jsonl' with your desired output filename
convert_df_to_jsonl(final_result, '/data/dataset/kefu/bge_finetune_emb_from_db2.jsonl', 
                    query="question", pos_col="answer_positive", neg_col="answer_negative")

In [276]:
tokenizer = AutoTokenizer.from_pretrained(
    "/workspace/data/private/zhuxiaohai/models/bge_emb_finetune_from_db",
    use_fast=False,
)

In [277]:
a = tokenizer(final_result.answer.tolist())

In [278]:
len_list = []
for i in a["input_ids"]:
    len_list.append(len(i))
max(len_list)

969

In [279]:
sum(len_list)/len(len_list)

109.28571428571429

In [280]:
for index, i in enumerate(len_list):
    if i > 900:
        print(index)

1696
2291
2292
2293
2294


In [281]:
final_result.answer.tolist()[2294]

'我们在持续优化烘干效果，请您连接手机APP，升级到最新版本后再体验\n- 小件与大件（如床单与上衣），很多衣物混合在一起洗，导致的烘不干\n- 您好，很抱歉给您带来不便，小件衣物与床单等大件衣物一起洗烘，大件衣物会把小件衣物包裹住，使洗衣机烘干时的抖散动作无法展开，从而导致小件衣物无法烘干，建议您分类洗烘，使用【家纺】+【洗烘】程序，单独洗烘四件套或窗帘等大件织物。\n- 用户所处的环境在沿海潮湿的地带或梅雨季节，烘干后的衣物放置在洗衣机内长时间未取出，导致的潮湿\n- 您好，很抱歉给您带来不便，如您所处的环境在沿海潮湿的地带或梅雨季节，烘干后未能及时取出，可能会出现微微潮湿，建议您再使用【快速】+【烘干】程序继续烘一下且及时拿出即可。\n- 用户放入的衣物过多，挤满内筒，导致的烘不干\n- 您好，很抱歉给您带来不便，如果您在滚筒内放入了过多衣物，充满了整个滚筒。使洗衣机烘干时的抖散动作无法展开，部分衣物因筒内空间不足而导致烘干效果不佳，建议您洗烘时减少衣物投放量，选择【快速】+【烘干】模式继续烘一会儿。\n- 正常使用，但有衣物的边角没能烘干\n- 您好，很抱歉给您带来不便，石头分子筛洗烘一体机H1的洗烘或烘干程序，会根据您选择的洗护程序与和RR智能算法专门调试的，软件算法自动判干，衣干即停。不同材质及重量的衣物含水率不同，混合洗烘时可能会出现烘干不匀的情况，您可以选择【快速】+【烘干】继续烘干即可，建议您下次分类洗涤或减少衣物量。\n- 放入洗衣袋内的衣物，没能烘干\n- 您好，很抱歉给您带来不便，衣物放入洗衣袋内，会影响衣物舒展而导致烘干效果不佳，使用烘干功能时，可以将衣物取出烘干。建议您将单件衣物放入单个洗衣袋内使用，避免多件衣物放入同一个袋内而影响烘干效果。\n- 【干衣度】上选择了【微干】而衣物有潮湿感\n- 您好，很抱歉给您带来不便，您在使用烘干程序的时候，干衣度选择了【微干】烘干后的衣物是略带水分，【微干】的潮湿效果适合熨烫定型使用。建议您下次将【干衣度】调节成【普通】或者【超干】\n- 手洗的衣物，如没有彻底拧干而选择了【烘干】模式，导致烘不干\n- 您好，很抱歉给您带来不便，您手洗的衣物，如没有彻底拧干而选择了【烘干】模式，可能会导致烘干效果不佳，建议使用【洗烘】+【单脱水】，甩干后再进行烘干。'